Домашнее задание 1

Использовать dataset MovieLens

Построить рекомендации (регрессия, предсказываем оценку) на фичах:

TF-IDF на тегах и жанрах

Средние оценки (+ median, variance, etc.) пользователя и фильма

Оценить RMSE на тестовой выборке

In [1]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [7]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [8]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [9]:

np.mean(ratings[ratings['movieId'] == 193609].rating)

4.0

Готовим датасет со средними оценками пользователя и  фильма:

In [10]:

movieId_ratings = pd.DataFrame()
print(max(ratings['movieId']))
movieId = []
rating = []
for i in tqdm(range(1, 193610)):
    var = np.mean(ratings[ratings['movieId'] == i].rating)
    if var != 'Nan':
        movieId.append(i)
        rating.append(var)
movieId_ratings['movieId'] = movieId
movieId_ratings['ratings'] = rating

193609


In [13]:
movieId_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9724 entries, 0 to 193608
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9724 non-null   int64  
 1   ratings  9724 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 227.9 KB


In [12]:
movieId_ratings.dropna(inplace=True)

In [14]:
movies_with_ratings = movies.join(movieId_ratings.set_index('movieId'), on='movieId')

In [15]:

movies_with_ratings.dropna(inplace=True)
movies_with_ratings

,movieId,title,genres,ratings
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.920930
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.431818
2,3,Grumpier Old Men (1995),Comedy|Romance,3.259615
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.357143
4,5,Father of the Bride Part II (1995),Comedy,3.071429
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,4.000000
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,3.500000
9739,193585,Flint (2017),Drama,3.500000
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,3.500000


Строим модель рекомендации (регрессия, предсказываем оценку) на фичах:

In [16]:

from sklearn.model_selection import train_test_split

In [17]:
def change_string(s):
    return " ".join(s.replace(' ', '').replace('-','').split('|'))

In [18]:
movie_genres = [change_string(g) for g in movies_with_ratings.genres.values]

In [19]:

count_vect = CountVectorizer()

In [20]:

X_train_counts = count_vect.fit_transform(movie_genres)

In [21]:

X_train_counts.toarray()

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [22]:

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [23]:
y = movies_with_ratings['ratings']

In [24]:
X_test, X_train, y_test, y_train = train_test_split(X_train_tfidf, y, test_size=0.9, random_state = 1000)

In [25]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [26]:
X_train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.54781407, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.56455748, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [27]:
model =  LinearRegression().fit(X_train, y_train)

In [29]:
y_pred = model.predict(X_test)

In [30]:

len(y_test)

972

In [32]:
#Оцениваем RMSE

In [31]:
print("RMSE =",np.sqrt(np.mean((y_test-y_pred)**2)))

RMSE = 0.8150125644925946
